In [ ]:
import functools
import os

import jax
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from src.rts.config import EnvConfig
from src.rts.env import Board, EnvState, init_state, random_step, p1_step
from src.rts.utils import assert_valid_state, get_legal_moves
from src.rts.visualizaiton import visualize_board
from src.rl.pqn import Params

In [ ]:
from flax import nnx
from src.rl.pqn import Model
import optax

In [ ]:
os.environ["JAX_CHECK_TRACER_LEAKS"] = "TRUE"

## Random play

In [ ]:
# Play a game with random moves for 1000 steps
# Visualize the board interactivly
rng_key = jax.random.PRNGKey(3)
config = EnvConfig(
    num_players=4,
    board_width = 10,
    board_height = 10,
    num_neutral_bases = 6,
    num_neutral_troops_start = 10,
    neutral_troops_min = 4,
    neutral_troops_max = 10,
    player_start_troops=5,
    bonus_time=10,
)
state = init_state(rng_key, config)

In [ ]:
for i in range(10):
    rng_key, subkey = jax.random.split(rng_key)
    state = random_step(state, subkey, config)
    assert_valid_state(state)
    if i % 1 == 0:
        visualize_board(state)

## Batched Random

In [ ]:
@functools.partial(jax.jit, static_argnames=("config",))
def batched_step(states, rng_keys, config):
    def single_step(state, key):
        return random_step(state, key, config)

    return jax.vmap(single_step)(states, rng_keys)

In [ ]:
# Now we vmap
N = 50
rng_key = jax.random.PRNGKey(3)
rng_keys = jax.random.split(rng_key, N)

# Create the initial state for each game via vmap.
batched_init_state = jax.vmap(lambda key: init_state(key, config))
states = batched_init_state(rng_keys)

In [ ]:
for i in range(50):
    # For each parallel game, split its RNG key into two:
    # keys_split will have shape (N, 2, key_shape).
    keys_split = jax.vmap(lambda key: jax.random.split(key, 2))(rng_keys)
    # Update rng_keys to the first half and use the second half as subkeys.
    rng_keys = keys_split[:, 0]
    subkeys = keys_split[:, 1]

    # Take one step in parallel for all games.
    states = batched_step(states, subkeys, config)

    # Visualize and validate
    if i % 250 == 0:
        board = Board(
            player_troops = states.board.player_troops[79],
            neutral_troops = states.board.neutral_troops[79],
            bases = states.board.bases[79],
        )
        single_state = EnvState(board = board)
        assert_valid_state(single_state)
        visualize_board(single_state)

## Run with Neural Network

In [ ]:
rng_key = jax.random.PRNGKey(3)
width = 10
height = 10
config = EnvConfig(
    num_players=4,
    board_width = width,
    board_height = height,
    num_neutral_bases = 3,
    num_neutral_troops_start = 5,
    neutral_troops_min = 4,
    neutral_troops_max = 10,
    player_start_troops=5,
    bonus_time=10,
)
model = Model(width*height*(config.num_players + 2), 256, width*height*4, rngs=nnx.Rngs(0))
nnx.display(model)

In [ ]:
state = init_state(rng_key, config)
for i in range(50):
    flat_state = jnp.array(state.board.flatten())
    legal_mask = get_legal_moves(state, 0)
    legal_mask = jnp.array(legal_mask.flatten())
    action = jnp.argmax((model(flat_state) + 10) * legal_mask)
    # split action from int to array
    # y, x, direction
    action = jnp.array([action // (width*4), (action % (width*4))//4, action % 4])
    rng_key, subkey = jax.random.split(rng_key)
    state, p1_reward = p1_step(state, subkey, config, action)
    assert_valid_state(state)
    if i % 5 == 0:
        visualize_board(state)
    print(p1_reward)

## PQN

In [ ]:
flat_state = state.board.flatten()
legal_mask = get_legal_moves(state, 0).flatten()
q_net_action = jnp.argmax((model(flat_state) + 1000) * legal_mask)
q_net_action

In [ ]:
from src.rl.pqn import single_rollout
params = Params(
    num_iterations=100,
    lr=2e-4,
    gamma=0.75,
    q_lambda=0.95,
    num_envs=2048,
    num_steps=250,
    update_epochs=3,
    num_minibatches=10,
    epsilon=0.09,
)
q_net = Model(width*height*(config.num_players + 2), 256, width*height*4, rngs=nnx.Rngs(0))
optimizer = nnx.Optimizer(q_net, optax.adam(params.lr))

obs_buffer, actions_buffer, rewards_buffer, done_buffer, next_obs_buffer, cum_return = single_rollout(rng_key, config, q_net, params.num_steps, params.epsilon)
actions_buffer, done_buffer, rewards_buffer

In [ ]:
from src.rl.pqn import q_lambda_return

returns = q_lambda_return(q_net, rewards_buffer, done_buffer, next_obs_buffer, params.gamma, params.q_lambda)
returns

In [ ]:
from src.rl.pqn import train_step

train_step(q_net, optimizer, obs_buffer, actions_buffer, returns)

print(jnp.take_along_axis(q_net(obs_buffer), actions_buffer[:, None], axis=1).squeeze())
print(returns)

In [ ]:
def train(q_net: Model, optimizer: nnx.Optimizer, config: EnvConfig, params: Params):
    rng_key = jax.random.PRNGKey(0)
    losses = []
    for iteration in range(params.num_iterations):
        rng_key, rollout_key = jax.random.split(rng_key)
        rollout = single_rollout(rollout_key, config, q_net, params.num_steps, params.epsilon)
        obs_buffer, actions_buffer, rewards_buffer, done_buffer, next_obs_buffer, cum_return = rollout

        returns = q_lambda_return(q_net, rewards_buffer, done_buffer, next_obs_buffer, params.gamma, params.q_lambda)

        for epoch in range(params.update_epochs):
            loss = train_step(q_net, optimizer, obs_buffer, actions_buffer, returns)
            losses.append(loss)
        if iteration % 10 == 0:
            print(f"Iteration {iteration} - Loss: {loss}")

    # evaluate
    state = init_state(rng_key, config)
    q_net(state.board.flatten())

    plt.plot(losses)

    return q_net

In [ ]:
width = 10
height = 10
config = EnvConfig(
    num_players=2,
    board_width = width,
    board_height = height,
    num_neutral_bases = 3,
    num_neutral_troops_start = 5,
    neutral_troops_min = 4,
    neutral_troops_max = 10,
    player_start_troops=5,
    bonus_time=10,
)
params = Params(
    num_iterations=100,
    lr=2e-4,
    gamma=0.75,
    q_lambda=0.95,
    num_envs=2048,
    num_steps=250,
    update_epochs=3,
    num_minibatches=10,
    epsilon=0.09,
)
q_net = Model(width*height*4, 256, width*height*4, rngs=nnx.Rngs(0))
optimizer = nnx.Optimizer(q_net, optax.adam(params.lr))

q_net = train(q_net, optimizer, config, params)

In [ ]:
# Now we can use the trained model to play a game
state = init_state(rng_key, config)
rewards = []
for i in range(200):
    legal_mask = get_legal_moves(state, 0)
    legal_mask = jnp.array(legal_mask.flatten())
    print(q_net(state.board.flatten())* legal_mask)
    action = jnp.argmax((q_net(state.board.flatten()) + 1000) * legal_mask)
    # split action from int to array
    # y, x, direction
    action = jnp.array([action // (width*4), (action % (width*4))//4, action % 4])
    rng_key, subkey = jax.random.split(rng_key)
    state, p1_reward = p1_step(state, subkey, config, action)
    rewards.append(p1_reward)
    assert_valid_state(state)
    if i % 5 == 0:
        visualize_board(state)
# for each step print the cumulative reward to the end from that step
print(np.cumsum(rewards))
plt.plot(np.cumsum(rewards))

## PQN with vmap

### Minibatched

In [ ]:
from src.rl.pqn import train_minibatched

width = 10
height = 10
config = EnvConfig(
    num_players=2,
    board_width = width,
    board_height = height,
    num_neutral_bases = 3,
    num_neutral_troops_start = 5,
    neutral_troops_min = 4,
    neutral_troops_max = 10,
    player_start_troops=5,
    bonus_time=10,
)
params = Params(
    num_iterations=2000,
    lr=2e-4,
    gamma=0.75,
    q_lambda=0.95,
    num_envs=512,
    num_steps=250,
    update_epochs=3,
    num_minibatches=10,
    epsilon=0.09,
)
q_net = Model(width*height*4, 512, width*height*4, rngs=nnx.Rngs(0))
optimizer = nnx.Optimizer(q_net, optax.adam(params.lr))

q_net, losses, cum_returns = train_minibatched(q_net, optimizer, config, params)

plt.plot(losses)
plt.title("Losses")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.show()

plt.plot(cum_returns)
plt.title("Cumulative Returns")
plt.xlabel("Iteration")
plt.ylabel("Cumulative Return")
plt.show()

In [ ]:
plt.plot(np.mean(cum_returns, axis=0))

In [ ]:
# Now we can use the trained model to play a game
rng_key = jax.random.PRNGKey(3)
state = init_state(rng_key, config)
rewards = []
for i in range(300):
    legal_mask = get_legal_moves(state, 0)
    legal_mask = jnp.array(legal_mask.flatten())
    action = jnp.argmax((q_net(state.board.flatten()) + 1000) * legal_mask)
    # split action from int to array
    # y, x, direction
    action = jnp.array([action // (width*4), (action % (width*4))//4, action % 4])
    rng_key, subkey = jax.random.split(rng_key)
    state, p1_reward = p1_step(state, subkey, config, action)
    rewards.append(p1_reward)
    visualize_board(state)
# for each step print the cumulative reward to the end from that step
plt.plot(np.cumsum(rewards))